In [1]:
WORKER_COUNT = 8

In [2]:
# whether to log each feature and sequence status
verbose = True

In [3]:
import gc
import os
import pandas as pd
pd.options.display.max_rows = 5000
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.append('..')

In [4]:
# setup paths
pwd = os.getcwd().replace("notebooks","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [5]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s, [%(levelname)-8s] [%(filename)s:%(lineno)d] %(message)s',
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()

In [6]:
from deepvideoclassification.pretrained_CNNs import pretrained_model_names, pretrained_model_names_bucketed


Using TensorFlow backend.


# Create list of experiments to be run

* batch 1 = run frozen image MLP, LRCNNs and concat models on 1 of each pretrained_model_name in buckets (bucketed on feature sizes and limited to max sequence_length of 10)

* batch 2 = for best configurations from batch 1, run other pretrained models in buckets and run longer sequence lengths

* batch 3 = run trainable MLP and LRCNN on best performing frozen variants

* batch 4 = run trainable but initializing with best CNN weights

* batch 5 = run C3D models

* batch 6 = analyze effect of dropout and pooling with best model

# Batch 1

In [7]:
experiment_batch_name = 'experiment_batch_1'

In [8]:
# init model id - need to make sure we pick up where we leave off don't overwrite it between batches
model_id_start = 0

In [9]:
# init list of experiments
experiments = []

In [10]:
pooling = 'max'
layer_sizes = [512, 256, 128, 0]
dropouts = [0.2]
sequence_lengths = [3,5,10]
sequence_models = ["LSTM", "SimpleRNN", "GRU", "Convolution1D"]
sequence_model_layer_counts = [1,2]

In [11]:
####################
### image_MLP_frozen 
####################

for pretrained_model_name in pretrained_model_names_bucketed:
    for layer_1_size in layer_sizes:
        for layer_2_size in layer_sizes:
            for layer_3_size in layer_sizes:
                for dropout in dropouts:

                    # build experiment parameters
                    experiment = {}
                    
                    experiment['architecture'] = 'image_MLP_frozen'
                    experiment['sequence_length'] = 1
                    experiment['pretrained_model_name'] = pretrained_model_name
                    experiment['layer_1_size'] = layer_1_size
                    experiment['layer_2_size'] = layer_2_size
                    experiment['layer_3_size'] = layer_3_size
                    experiment['dropout'] = dropout
                    experiment['pooling'] = 'max' # outperforms avg across all parameters
                    
                    # add to list of experiments
                    experiments.append(experiment)

In [12]:
####################
### video_MLP_concat
####################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:

                        # build experiment parameters
                        experiment = {}

                        experiment['architecture'] = 'video_MLP_concat'
                        experiment['pretrained_model_name'] = pretrained_model_name
                        experiment['layer_1_size'] = layer_1_size
                        experiment['layer_2_size'] = layer_2_size
                        experiment['layer_3_size'] = layer_3_size
                        experiment['dropout'] = dropout
                        experiment['pooling'] = 'max' # outperforms avg across all parameters
                        experiment['sequence_length'] = sequence_length

                        # add to list of experiments
                        experiments.append(experiment)

In [13]:
######################
### video_LRCNN_frozen
######################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:
                        for sequence_model in sequence_models:
                            for sequence_model_layers in sequence_model_layer_counts:

                                # build experiment parameters
                                experiment = {}

                                experiment['architecture'] = 'video_LRCNN_frozen'
                                experiment['pretrained_model_name'] = pretrained_model_name
                                experiment['layer_1_size'] = layer_1_size
                                experiment['layer_2_size'] = layer_2_size
                                experiment['layer_3_size'] = layer_3_size
                                experiment['dropout'] = dropout
                                experiment['pooling'] = 'max' # outperforms avg across all parameters
                                experiment['sequence_model'] = sequence_model
                                experiment['sequence_model_layers'] = sequence_model_layers
                                experiment['sequence_length'] = sequence_length

                                # add to list of experiments
                                experiments.append(experiment)

In [14]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)

### create model id column for this experiment batch
model_id_list = list(range(0,len(experiments)))
experiments['model_id'] = model_id_list

# assign to workers
experiments['WORKER'] = experiments['model_id'].apply(lambda x: x % WORKER_COUNT)

In [53]:
##################################
### output experiment batch to CSV
##################################
print(experiment_batch_name)
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

# Batch 2

In [39]:
# run other pretrained models for best configurations from batch 1
# and run longer sequence lengths

In [40]:
experiment_batch_name = 'experiment_batch_2'

In [41]:
# init model id - need to make sure we pick up where we leave off don't overwrite it between batches
model_id_start = pd.read_csv(pwd + "experiments/experiment_batch_1.csv")['model_id'].max() + 1

In [42]:
# init list of experiments
experiments = []

In [ ]:
# TODO

In [43]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)

### create model id column for this experiment batch
model_id_list = list(range(0,len(experiments)))
experiments['model_id'] = model_id_list

# assign to workers
experiments['WORKER'] = experiments['model_id'].apply(lambda x: x % WORKER_COUNT)

In [ ]:
##################################
### output experiment batch to CSV
##################################
print(experiment_batch_name)
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

# Batch 3

In [ ]:
# run trainable MLP and LRCNN on best performing frozen variants

In [ ]:
#######################
### image_MLP_trainable
#######################

architecture = 'video_LRCNN_trainable'

In [ ]:
#########################
### video_LRCNN_trainable
#########################

architecture = 'video_LRCNN_trainable'

# Batch 4

In [ ]:
# run trainable but initializing with best CNN weights

# Batch 5

In [ ]:
#######
### C3D
#######

architecture = 'C3D'

############
### C3Dsmall
############

architecture = 'C3Dsmall' 

# Analyze results

## load results.json for all models into dataframe

In [21]:
path_models = pwd + 'models/'

results = []

for folder, subs, files in os.walk(path_models):
    for filename in files:
        if 'results.json' in filename:
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)        
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [22]:
results.head(10).T

,317,153,344,182,273,269,118,293,325,240
architecture,video_mlp_concat,video_mlp_concat,video_mlp_concat,video_mlp_concat,image_mlp_frozen,image_mlp_frozen,image_mlp_frozen,video_mlp_concat,image_mlp_frozen,image_mlp_frozen
batch_size,32,32,32,32,32,32,32,32,32,32
convolution_kernel_size,3,3,3,3,3,3,3,3,3,3
data_total_rows_test,3137,3137,3137,3137,3139,3139,3139,3137,3139,3139
data_total_rows_train,60553,60553,60553,60553,60597,60597,60597,60553,60597,60597
data_total_rows_valid,6412,6412,6412,6412,6416,6416,6416,6412,6416,6416
dropout,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
fit_best_round,1,1,2,1,2,2,2,1,3,2
fit_dt_test_duration_seconds,0,1,0,2,2,1,1,0,1,1
fit_dt_test_end,2019-01-20 20:46:20,2019-01-20 21:06:55,2019-01-20 20:54:33,2019-01-20 16:29:10,2019-01-20 14:25:08,2019-01-20 14:01:02,2019-01-20 14:07:05,2019-01-20 20:50:39,2019-01-20 13:54:52,2019-01-20 14:13:38


## Merge done status onto experiments

In [23]:
experiments = pd.merge(experiments, results[['model_id','fit_val_acc']], left_on='model_id', right_on='model_id', how='left')
experiments['done'] = (experiments['fit_val_acc']>0).astype(int)
del experiments['fit_val_acc']

In [24]:
experiments.head()

,architecture,dropout,layer_1_size,layer_2_size,layer_3_size,pooling,pretrained_model_name,sequence_length,sequence_model,sequence_model_layers,model_id,WORKER,done
0,image_MLP_frozen,0.2,512,512,512,max,inception_resnet_v2,1,NaN,NaN,0,0,1
1,image_MLP_frozen,0.2,512,512,256,max,inception_resnet_v2,1,NaN,NaN,1,1,1
2,image_MLP_frozen,0.2,512,512,128,max,inception_resnet_v2,1,NaN,NaN,2,2,1
3,image_MLP_frozen,0.2,512,512,0,max,inception_resnet_v2,1,NaN,NaN,3,3,1
4,image_MLP_frozen,0.2,512,256,512,max,inception_resnet_v2,1,NaN,NaN,4,4,1


In [25]:
print("{}/{} experiments done".format(experiments[experiments['done'] == 1].shape[0], len(experiments)))

363/5376 experiments done


## total experiments, split by architecture

In [29]:
experiments['architecture'].value_counts()

video_LRCNN_frozen    4608
video_MLP_concat       576
image_MLP_frozen       192
Name: architecture, dtype: int64

## analyze remaining experiments, split on architecture

In [26]:
experiments[experiments['done']==0]['architecture'].value_counts()

video_LRCNN_frozen    4608
video_MLP_concat       405
Name: architecture, dtype: int64

In [27]:
experiments[experiments['architecture'] == 'video_MLP_concat']

,architecture,dropout,layer_1_size,layer_2_size,layer_3_size,pooling,pretrained_model_name,sequence_length,sequence_model,sequence_model_layers,model_id,WORKER,done
192,video_MLP_concat,0.2,512,512,512,max,inception_resnet_v2,3,NaN,NaN,192,0,1
193,video_MLP_concat,0.2,512,512,256,max,inception_resnet_v2,3,NaN,NaN,193,1,1
194,video_MLP_concat,0.2,512,512,128,max,inception_resnet_v2,3,NaN,NaN,194,2,1
195,video_MLP_concat,0.2,512,512,0,max,inception_resnet_v2,3,NaN,NaN,195,3,1
196,video_MLP_concat,0.2,512,256,512,max,inception_resnet_v2,3,NaN,NaN,196,4,1
197,video_MLP_concat,0.2,512,256,256,max,inception_resnet_v2,3,NaN,NaN,197,5,1
198,video_MLP_concat,0.2,512,256,128,max,inception_resnet_v2,3,NaN,NaN,198,6,1
199,video_MLP_concat,0.2,512,256,0,max,inception_resnet_v2,3,NaN,NaN,199,7,1
200,video_MLP_concat,0.2,512,128,512,max,inception_resnet_v2,3,NaN,NaN,200,0,1
201,video_MLP_concat,0.2,512,128,256,max,inception_resnet_v2,3,NaN,NaN,201,1,1


In [28]:
experiments[experiments['architecture'] == 'video_MLP_concat'].sort_values("sequence_length").head()

,architecture,dropout,layer_1_size,layer_2_size,layer_3_size,pooling,pretrained_model_name,sequence_length,sequence_model,sequence_model_layers,model_id,WORKER,done
192,video_MLP_concat,0.2,512,512,512,max,inception_resnet_v2,3,NaN,NaN,192,0,1
314,video_MLP_concat,0.2,0,128,128,max,vgg16,3,NaN,NaN,314,2,1
315,video_MLP_concat,0.2,0,128,0,max,vgg16,3,NaN,NaN,315,3,1
316,video_MLP_concat,0.2,0,0,512,max,vgg16,3,NaN,NaN,316,4,1
317,video_MLP_concat,0.2,0,0,256,max,vgg16,3,NaN,NaN,317,5,1


# Copy experiment files to s3

In [ ]:
model_ids = list(results['model_id'])
model_ids.sort()

In [ ]:
for i, model_id in enumerate(model_ids):
    
    path_model = pwd + '/models/' + str(model_id) + '/'

    # aws s3 ls on path returns 0 if it exists so check if doesn't exist, then sync
    if os.system("aws s3 ls s3://thesisvids/penguins/models/" + str(model_id) + "/") > 0:
        print("Synching {}/{} - model_id={}".format(i+1,len(model_ids),model_id))
        response = os.system("aws s3 sync " + path_model + " s3://thesisvids/penguins/models/" + str(model_id) + "/")
        if response != 0:
            print("ERROR syncing model_id = {}".format(model_id))
    else:
        print("Already synched {}/{} - model_id={}".format(i+1,len(model_ids),model_id))

# XXX

In [30]:
experiments

,architecture,dropout,layer_1_size,layer_2_size,layer_3_size,pooling,pretrained_model_name,sequence_length,sequence_model,sequence_model_layers,model_id,WORKER,done
0,image_MLP_frozen,0.2,512,512,512,max,inception_resnet_v2,1,NaN,NaN,0,0,1
1,image_MLP_frozen,0.2,512,512,256,max,inception_resnet_v2,1,NaN,NaN,1,1,1
2,image_MLP_frozen,0.2,512,512,128,max,inception_resnet_v2,1,NaN,NaN,2,2,1
3,image_MLP_frozen,0.2,512,512,0,max,inception_resnet_v2,1,NaN,NaN,3,3,1
4,image_MLP_frozen,0.2,512,256,512,max,inception_resnet_v2,1,NaN,NaN,4,4,1
5,image_MLP_frozen,0.2,512,256,256,max,inception_resnet_v2,1,NaN,NaN,5,5,1
6,image_MLP_frozen,0.2,512,256,128,max,inception_resnet_v2,1,NaN,NaN,6,6,1
7,image_MLP_frozen,0.2,512,256,0,max,inception_resnet_v2,1,NaN,NaN,7,7,1
8,image_MLP_frozen,0.2,512,128,512,max,inception_resnet_v2,1,NaN,NaN,8,0,1
9,image_MLP_frozen,0.2,512,128,256,max,inception_resnet_v2,1,NaN,NaN,9,1,1
